# Pilot Patient
## Import Body Map from Infopad csv file

#### Last updated on : 3rd Oct 2020  -  Amar Jaiswal

In [1]:
es_base_url = 'http://localhost:9200'
headers = {'Content-Type': 'application/json'}

In [2]:
# auto sugestion with tab
%config IPCompleter.greedy=True
%config InlineBackend.figure_format = 'retina'
%matplotlib inline 

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns

import datetime as datetime

import requests
import json

import os
os.getcwd()

'/lhome/amardj/ISM/python_code/es_scripts/pilot_scripts'

In [4]:
sys_date = datetime.datetime.today().strftime( '%Y%m%d')
sys_date = f'{sys_date}' 
sys_date

'20201026'

In [5]:
data_dir = f'./data/{sys_date}/'

In [6]:
body_main_file = f'{data_dir}body_map.csv'

In [7]:
raw_csv = pd.read_csv( body_main_file, delimiter=';')

In [9]:
raw_csv.head()

,Brukernavn,Journal ID,Fødselsår,Forside 0 0,Forside 0 1,Forside 0 2,Forside 0 3,Forside 0 4,Forside 0 5,Forside 0 6,...,Bakside 7 5,Bakside 7 6,Bakside 7 7,Bakside 7 8,Bakside 7 9,Bakside 7 10,Bakside 7 11,Bakside 7 12,Bakside 7 13,Bakside 7 14
0,support42,3002,1990,,,,,,,,...,,,,,,,,,,
1,support27,1005,1961,,,,,,,,...,,,,,,,,,,
2,support27,1006,1957,,,,,,,,...,,,,,,,,,,


In [10]:
raw = raw_csv.rename(columns={'Fødselsår': 'birthYear', 'Brukernavn':'clinicId', 'Journal ID':'journalId'})

In [11]:
exclude_patients_from_clinic = raw[
    (raw.clinicId != 'Fredrik Granviken') 
    & (raw.clinicId != 'Anita Formo Bones')
    & (raw.clinicId != 'Ingebrigt Meisingset')
    & (raw.clinicId != 'Testinstitutt')
    & (raw.clinicId != 'support32')
    & (raw.clinicId != 'support3')
] #"Testinstitutt", "support32","support3"

In [12]:
exclude_patients_from_clinic.shape #(*, 243)

(3, 243)

In [13]:
exclude_patients_from_clinic.head()

,clinicId,journalId,birthYear,Forside 0 0,Forside 0 1,Forside 0 2,Forside 0 3,Forside 0 4,Forside 0 5,Forside 0 6,...,Bakside 7 5,Bakside 7 6,Bakside 7 7,Bakside 7 8,Bakside 7 9,Bakside 7 10,Bakside 7 11,Bakside 7 12,Bakside 7 13,Bakside 7 14
0,support42,3002,1990,,,,,,,,...,,,,,,,,,,
1,support27,1005,1961,,,,,,,,...,,,,,,,,,,
2,support27,1006,1957,,,,,,,,...,,,,,,,,,,


In [14]:
es = []

for index, df in exclude_patients_from_clinic.iterrows():
    #print(index)

    
    df_1_row = df.replace(' ',np.nan).dropna(axis=0,how="all")
    
    patient_id = df_1_row[0].replace(' ','_').lower() + '_'+str(df_1_row[1]) +'_'+str(df_1_row[2])
            

    print(patient_id)

    patient = {}
    patient_details = {}
    body_map = {}

    patient.update({'patientId': patient_id}) 

    front = []
    back = []

    for val in df_1_row.index[3:].tolist():
        tokens = str(val).split(' ')

        if tokens[0] == 'Forside':     
            front.append({ 'x':int(tokens[1]), 'y':int(tokens[2])})

        if tokens[0] == 'Bakside':
            back.append({ 'x':int(tokens[1]), 'y':int(tokens[2])})

    body_map.update({'front':front})
    body_map.update({'back':back})

    patient_details.update({'body_map':body_map})
    patient.update({'patientDetails':patient_details})

    es.append(patient)
    
es;

support42_3002_1990
support27_1005_1961
support27_1006_1957


In [15]:
json.dumps(es)

'[{"patientId": "support42_3002_1990", "patientDetails": {"body_map": {"front": [{"x": 2, "y": 5}, {"x": 3, "y": 2}, {"x": 4, "y": 2}, {"x": 5, "y": 5}], "back": [{"x": 3, "y": 1}, {"x": 3, "y": 2}, {"x": 3, "y": 3}, {"x": 3, "y": 6}, {"x": 4, "y": 1}, {"x": 4, "y": 2}, {"x": 4, "y": 3}, {"x": 4, "y": 6}]}}}, {"patientId": "support27_1005_1961", "patientDetails": {"body_map": {"front": [{"x": 1, "y": 7}, {"x": 3, "y": 10}, {"x": 3, "y": 13}, {"x": 4, "y": 10}, {"x": 4, "y": 13}, {"x": 6, "y": 7}], "back": [{"x": 1, "y": 7}, {"x": 3, "y": 10}, {"x": 3, "y": 13}, {"x": 4, "y": 7}, {"x": 4, "y": 10}, {"x": 4, "y": 13}, {"x": 6, "y": 7}]}}}, {"patientId": "support27_1006_1957", "patientDetails": {"body_map": {"front": [{"x": 6, "y": 9}, {"x": 7, "y": 9}], "back": [{"x": 0, "y": 9}, {"x": 1, "y": 9}, {"x": 1, "y": 12}]}}}]'

In [16]:
with open(f'{data_dir}body_map.json', 'w', encoding='utf-8') as f:
    json.dump(es, f, ensure_ascii=False, indent=0)

# 
# 
# 

# Elasticsearch Operations

## Code for getting a patient by patientId from the ES

# 
## Set the elasticsearch (es) index name
## 

In [17]:
es_index = 'manual_patients'

# 

In [18]:
def get_all_docs(es_index=es_index):
    response = requests.post(es_base_url+'/'+es_index+'/_search?pretty', headers=headers, data='{}')
    return response.json()

In [19]:
def get_doc_by_pid(pid, es_index=es_index,):
    data = '{  "query": {    "match": {      "patientId": "'+pid+'"    }  }}'
    response = requests.post(es_base_url+'/'+es_index+'/_search?pretty', headers=headers, data=data)
    return response.json()

In [20]:
def get_id_for_pid( pid, es_index=es_index):
    params = (('filter_path', 'hits.hits._source'))
    data = '{  "query": {    "match": {      "patientId": "'+pid+'"    }  }}'
    response = requests.get(f'{es_base_url}/{es_index}/_search?filter_path=hits.hits._id', headers=headers, data=data)
    response_json = response.json()
    return response_json.get('hits', {}).get('hits', [{}])[0].get('_id')

In [21]:
get_id_for_pid('vidar_guttormsen_30160_1992')

# 

# 

## Code for getting only the list of patient ids from the ES

In [22]:
def update_es_doc_for_id( _id, json_doc, es_index=es_index):
    upsert_doc = json.dumps({'doc': json_doc, 'doc_as_upsert': True})
    response = requests.post(f'{es_base_url}/{es_index}/_update/{_id}?pretty', headers=headers, data=upsert_doc)
    return response.json()

In [23]:
def upadate_es_index(body_maps_list=es, es_index=es_index):
    for body_map in body_maps_list:
        pid = body_map.get('patientId')
        print(pid)
        _id = get_id_for_pid( pid, es_index)
        print(_id)
        
        if _id is None:
            print('_id was None')
        else :
            print('Upserting the pid:',pid, ' and _id: ', _id)
            update_es_doc_for_id(_id, body_map, es_index)

In [24]:
upadate_es_index( es, 'manual_patients')

support42_3002_1990
QBcdZHUBPCTLibsEh2Di
Upserting the pid: support42_3002_1990  and _id:  QBcdZHUBPCTLibsEh2Di
support27_1005_1961
PhcdZHUBPCTLibsEh2B8
Upserting the pid: support27_1005_1961  and _id:  PhcdZHUBPCTLibsEh2B8
support27_1006_1957
PxcdZHUBPCTLibsEh2Db
Upserting the pid: support27_1006_1957  and _id:  PxcdZHUBPCTLibsEh2Db
